In [1]:
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
from lohrasb.best_estimator import BaseModel
from sklearn.pipeline import Pipeline
from feature_engine.imputation import (
    CategoricalImputer,
    MeanMedianImputer
    )
from category_encoders import OrdinalEncoder
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    f1_score)
from sklearn.metrics import f1_score, make_scorer
from sklearn.linear_model import *
from sklearn.svm import *
from xgboost import *
from sklearn.linear_model import *
from lightgbm import *
from sklearn.neural_network import *
from imblearn.ensemble import *
from sklearn.ensemble import *


#### Example: Use Adult Data Set (a classification problem)
  
https://archive.ics.uci.edu/ml/datasets/Adult

#### Part 1: Use BestModel in sklearn pipeline


In [2]:
urldata= "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
# column names
col_names=["age", "workclass", "fnlwgt" , "education" ,"education-num",
"marital-status","occupation","relationship","race","sex","capital-gain","capital-loss","hours-per-week",
"native-country","label"
]
# read data
data = pd.read_csv(urldata,header=None,names=col_names,sep=',')
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


#### Define labels


In [3]:
data.loc[data['label']=='<=50K','label']=0
data.loc[data['label']==' <=50K','label']=0

data.loc[data['label']=='>50K','label']=1
data.loc[data['label']==' >50K','label']=1

data['label']=data['label'].astype(int)

#### Train test split

In [4]:
X = data.loc[:, data.columns != "label"]
y = data.loc[:, data.columns == "label"]


X_train, X_test, y_train, y_test =train_test_split(X, y, \
     test_size=0.33, stratify=y['label'], random_state=42)


#### Find feature types for later use

In [5]:
int_cols =  X_train.select_dtypes(include=['int']).columns.tolist()
float_cols =  X_train.select_dtypes(include=['float']).columns.tolist()
cat_cols =  X_train.select_dtypes(include=['object']).columns.tolist()


#### Define estimator and set its arguments 


In [6]:
estimator = XGBClassifier()
estimator_params = {
        "booster": ["gbtree","dart"],
        "eval_metric": ["auc"],
        "max_depth": [4, 5],
        "gamma": [0.1, 1.2],
        "subsample": [0.8],

    }
    

In [7]:

obj = BaseModel().optimize_by_gridsearchcv(
            estimator=estimator,
            estimator_params=estimator_params,
            fit_params = None,
            measure_of_accuracy=make_scorer(f1_score, greater_is_better=True),
            verbose=3,
            n_jobs=-1,
            random_state=42,
            cv=KFold(2),
        )

#### Build sklearn pipeline

In [8]:


pipeline =Pipeline([
            # int missing values imputers
            ('intimputer', MeanMedianImputer(
                imputation_method='median', variables=int_cols)),
            # category missing values imputers
            ('catimputer', CategoricalImputer(variables=cat_cols)),
            #
            ('catencoder', OrdinalEncoder()),
            # classification model
            ('obj', obj)

 ])


#### Run Pipeline

In [9]:
pipeline.fit(X_train,y_train)
y_pred = pipeline.predict(X_test)


Fitting 2 folds for each of 8 candidates, totalling 16 fits
[CV 1/2] END booster=gbtree, eval_metric=auc, gamma=0.1, max_depth=4, subsample=0.8;, score=0.695 total time=   1.9s
[CV 2/2] END booster=gbtree, eval_metric=auc, gamma=0.1, max_depth=4, subsample=0.8;, score=0.701 total time=   1.9s
[CV 1/2] END booster=gbtree, eval_metric=auc, gamma=0.1, max_depth=5, subsample=0.8;, score=0.692 total time=   0.5s
[CV 2/2] END booster=gbtree, eval_metric=auc, gamma=0.1, max_depth=5, subsample=0.8;, score=0.686 total time=   0.5s
[CV 1/2] END booster=gbtree, eval_metric=auc, gamma=1.2, max_depth=4, subsample=0.8;, score=0.691 total time=   0.4s
[CV 2/2] END booster=gbtree, eval_metric=auc, gamma=1.2, max_depth=4, subsample=0.8;, score=0.696 total time=   0.3s
[CV 1/2] END booster=gbtree, eval_metric=auc, gamma=1.2, max_depth=5, subsample=0.8;, score=0.687 total time=   0.4s
[CV 2/2] END booster=gbtree, eval_metric=auc, gamma=1.2, max_depth=5, subsample=0.8;, score=0.697 total time=   0.4s
[CV 

#### Check performance of the pipeline

In [10]:
print('F1 score : ')
print(f1_score(y_test,y_pred))
print('Classification report : ')
print(classification_report(y_test,y_pred))
print('Confusion matrix : ')
print(confusion_matrix(y_test,y_pred))


F1 score : 
0.709435534263695
Classification report : 
              precision    recall  f1-score   support

           0       0.90      0.94      0.92      8158
           1       0.77      0.66      0.71      2588

    accuracy                           0.87     10746
   macro avg       0.83      0.80      0.81     10746
weighted avg       0.87      0.87      0.87     10746

Confusion matrix : 
[[7648  510]
 [ 885 1703]]


#### Part 2:  Use BestModel as a standalone estimator 

In [11]:
X_train, X_test, y_train, y_test =train_test_split(X, y, \
     test_size=0.33, stratify=y['label'], random_state=42)

#### Transform features to make them ready for model input

In [12]:
transform_pipeline =Pipeline([
            # int missing values imputers
            ('intimputer', MeanMedianImputer(
                imputation_method='median', variables=int_cols)),
            # category missing values imputers
            ('catimputer', CategoricalImputer(variables=cat_cols)),
            #
            ('catencoder', OrdinalEncoder()),
            # classification model

 ])

#### Transform X_train and X_test

In [13]:
X_train=transform_pipeline.fit_transform(X_train,y_train)
X_test=transform_pipeline.transform(X_test)


#### Train model and predict

In [14]:
obj.fit(X_train,y_train)
y_pred = obj.predict(X_test)

Fitting 2 folds for each of 8 candidates, totalling 16 fits
[CV 1/2] END booster=gbtree, eval_metric=auc, gamma=0.1, max_depth=4, subsample=0.8;, score=0.695 total time=   0.5s
[CV 2/2] END booster=gbtree, eval_metric=auc, gamma=0.1, max_depth=4, subsample=0.8;, score=0.701 total time=   0.5s
[CV 1/2] END booster=gbtree, eval_metric=auc, gamma=0.1, max_depth=5, subsample=0.8;, score=0.692 total time=   0.4s
[CV 2/2] END booster=gbtree, eval_metric=auc, gamma=0.1, max_depth=5, subsample=0.8;, score=0.686 total time=   0.4s
[CV 1/2] END booster=gbtree, eval_metric=auc, gamma=1.2, max_depth=4, subsample=0.8;, score=0.691 total time=   0.4s
[CV 2/2] END booster=gbtree, eval_metric=auc, gamma=1.2, max_depth=4, subsample=0.8;, score=0.696 total time=   0.4s
[CV 1/2] END booster=gbtree, eval_metric=auc, gamma=1.2, max_depth=5, subsample=0.8;, score=0.687 total time=   0.4s
[CV 2/2] END booster=gbtree, eval_metric=auc, gamma=1.2, max_depth=5, subsample=0.8;, score=0.697 total time=   0.4s
[CV 

#### Check performance of the pipeline

In [15]:
print('F1 score : ')
print(f1_score(y_test,y_pred))
print('Classification report : ')
print(classification_report(y_test,y_pred))
print('Confusion matrix : ')
print(confusion_matrix(y_test,y_pred))

F1 score : 
0.709435534263695
Classification report : 
              precision    recall  f1-score   support

           0       0.90      0.94      0.92      8158
           1       0.77      0.66      0.71      2588

    accuracy                           0.87     10746
   macro avg       0.83      0.80      0.81     10746
weighted avg       0.87      0.87      0.87     10746

Confusion matrix : 
[[7648  510]
 [ 885 1703]]


In [16]:
obj.get_best_estimator()

XGBClassifier(base_score=0.5, booster='dart', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='auc', feature_types=None, gamma=1.2, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=4, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [17]:
obj.best_estimator

XGBClassifier(base_score=0.5, booster='dart', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='auc', feature_types=None, gamma=1.2, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=4, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0, ...)

#### Get fitted grid search object and its attributes

In [18]:
GridSearchObj = obj.get_optimized_object()
GridSearchObj.cv_results_

{'mean_fit_time': array([0.45011806, 0.42506862, 0.35904264, 0.42642343, 1.64516246,
        1.56979847, 1.46482694, 1.60262489]),
 'std_fit_time': array([0.00037694, 0.00108433, 0.00468564, 0.00417054, 0.03649056,
        0.01086044, 0.02808893, 0.18012309]),
 'mean_score_time': array([0.01332653, 0.0112164 , 0.01164138, 0.01162398, 0.14373004,
        0.14801002, 0.15070987, 0.15592504]),
 'std_score_time': array([0.00290859, 0.00098753, 0.00133955, 0.00120795, 0.00075591,
        0.00195098, 0.01791096, 0.01896286]),
 'param_booster': masked_array(data=['gbtree', 'gbtree', 'gbtree', 'gbtree', 'dart', 'dart',
                    'dart', 'dart'],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_eval_metric': masked_array(data=['auc', 'auc', 'auc', 'auc', 'auc', 'auc', 'auc', 'auc'],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dty